In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
chemin = "/content/drive/MyDrive/Stage Defense"


Mounted at /content/drive


In [ ]:
!pip install spacy-transformers
!python3 -m spacy download fr_dep_news_trf



     |████████████████████████████████| 2.3MB 8.4MB/s 
     |████████████████████████████████| 6.4MB 34.1MB/s 
     |████████████████████████████████| 1.0MB 36.1MB/s 
     |████████████████████████████████| 460kB 33.1MB/s 
     |████████████████████████████████| 3.3MB 31.5MB/s 
     |████████████████████████████████| 901kB 27.6MB/s 
     |████████████████████████████████| 10.1MB 196kB/s 
     |████████████████████████████████| 624kB 32.3MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
2021-07-12 11:55:30.486

In [ ]:


import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import gensim

In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
nlp = spacy.load('fr_dep_news_trf')
cheminmodeleW2V = "https://s3.us-east-2.amazonaws.com/embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin"
model_frWac = gensim.models.KeyedVectors.load_word2vec_format(cheminmodeleW2V, binary=True, unicode_errors="ignore")


In [ ]:
nltk.download('stopwords')
# stopwords de nltk
nltk_stopwords = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk_stopwords = nltk_stopwords |{"ni"}
"ni" in nltk_stopwords

True

In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
chemindata = chemin+"/Corpus/Toutes_mentions_Defense_annotees.csv"

# Pré-traitement corpus

In [ ]:
df = pd.read_csv(chemindata,sep=",", index_col=0, keep_default_na=False,na_values=["NA"])

In [ ]:
df = df[(df.cible_contexte != "")&(df.procédé_de_nommage!= "")]

In [ ]:
df_mention_w2v =  pd.DataFrame(liste_vecteurs_m,index=df.index)
df_contexte_w2v =  pd.DataFrame(liste_vecteurs_c,index=df.index)

mention

In [ ]:
def cree_vecteurs_designations(df):
  liste_vecteurs_m = []
  for mention in df.mention:
    doc = nlp(mention)
    if "défense" in mention.lower():
      print("BRUT",mention)
    lemmes = [t.lemma_ for t in doc if t.lemma_ not in nltk_stopwords|{"défense","defense"}]
    array_vec = np.array([model_frWac[lem] for lem in lemmes if (lem in model_frWac.vocab )])
    #print(array_vec.shape)
    if not array_vec.any():
      array_vec = np.zeros((1,500))
    array_moy = np.mean(array_vec,axis=0)
    liste_vecteurs_m.append(array_moy)
  return liste_vecteurs_m 


In [ ]:
liste_vecteurs_m = cree_vecteurs_designations(df)
df_mention_w2v =  pd.DataFrame(liste_vecteurs_m,index=df.index)

In [ ]:
df_w2v = df_mention_w2v

In [ ]:
 df_w2v.columns = list(range(len(df_w2v.columns)))

In [ ]:
df_w2v.to_csv(chemin+"/Corpus/W2V_mentions_Defense.csv",sep=",")

#Descripteurs Linguistiques fins

In [ ]:

import gensim
from sklearn.metrics.pairwise import cosine_similarity 
import re

In [ ]:
def moyenne_vecteurs(liste_mots):
  liste_vecteurs = []
  for mot in liste_mots:
    if mot in model_frWac.vocab:
      liste_vecteurs.append(model_frWac[mot])
  print(len(liste_vecteurs))
  return np.mean(np.array(liste_vecteurs),axis=0)

In [ ]:
def normalise(char):
  char = re.sub("[],\-\n;&]", " " , char)
  return char.lower()

In [ ]:
def normalise_mention(men):
  #men = men.lower()
  men = re.sub("^-","",men)
  men = men.replace("("," (")
  men = men.replace(")",")")
  return re.sub("\s+"," ",men).strip()

In [ ]:
def detecte_norme(mention):
  stopwords = nltk_stopwords | {"entre","vers","ni"}
  if "défense" in mention.lower() or "defense" in mention.lower():
    doc = nlp(normalise(mention))
    lemmes = [t.lemma_ for t in doc if t.lemma_ not in stopwords and re.match("\w+",t.lemma_)]
    #print(lemmes)
    if lemmes in  [["défense"],["defense"]]:
      return 1
  return 0

In [ ]:
def cos_sim(a,b):
  return cosine_similarity(a.reshape(1, -1),b.reshape(1, -1))[0][0]

In [ ]:
def contient_ordinal(mention):
  ordinal = re.compile("((derni)|(premi)((er)|(ère)))|(seconde?)|((deux|trois|quatri|cinqu|six|sept|huit|neuv|dix|vingt|trente|cent|mill)ième)")
  if ordinal.search(mention.lower()):
    return 1
  return 0

In [ ]:
def contient_superlatif(mention):
  set_superlatifs = {"plus ","meilleur ","moins ","pire ","favori","préféré","moindre"}
  if any(super in mention.lower() for super in set_superlatifs):
    return 1
  return 0

In [ ]:
def contient_lieu_geo(mention,vecteur_lieu,vecteur_geo):
  #if True or "laboratoire" not in mention and "mélange" not in mention: return 0,0,0,0,0
  #ajouter est type de lieu. Seuil = 0.30 Regarder tous les noms
  doc = nlp(mention)
  #print(doc[1].dep_)
  lieu,geo = 0,0
  sim_tete_lieu, sim_tete_geo,sim_tete_norme = 0,0,0
  for tok in doc:
    #print(tok,tok.pos_, tok.dep_, tok.tag_)
    lemme = tok.lemma_.lower()
    if lemme in model_frWac:
      sim_lieu = cos_sim(model_frWac[lemme],vecteur_lieu)
      sim_geo = cos_sim(model_frWac[lemme],vecteur_geo)
      if sim_geo > 0.40:
        
        geo = 1
      if sim_lieu > 0.40 and tok.pos_ in {"NOUN","PROPN"}:
        lieu = 1
        #print( tok.dep_, tok.pos_,sim_lieu,tok.lemma_, mention)
      #print(tok.dep_, tok.text)
      
      if (tok.dep_ == "ROOT" and all([tok.dep_ != "appos" for tok in doc ])) or tok.dep_ == "appos":
        #print(tok,tok.dep_, "|",mention)
        sim_tete_lieu = sim_lieu
        sim_tete_geo = sim_geo
        sim_tete_norme = cos_sim(model_frWac[lemme],model_frWac["défense"])

  if sim_tete_lieu == 0:
    pass
    print(mention, [t for t in doc if t.dep_ == "ROOT"])
  return lieu,geo, sim_tete_lieu, sim_tete_geo,sim_tete_norme


In [ ]:
def contient_guillemet(mention):

  return int('"' in mention or '»' in mention or '«' in mention)

In [ ]:
types_lieux = {"quartier","zone","lieu","espace","endroit","ville","localité","village","commune","agglomération","bourgade","cité","ghetto","arrondissement","terrain","surface"}
geos = {"france","europe","européen","hauts-de-seine","continental","continent","français","paris","parisien""européen","francilien","parisien","puteaux","nanterre","nanterrien","putéolien","courbevoie","courbevoisien","garenne-colombes","seine"}

In [ ]:
vecteur_geo = moyenne_vecteurs([mot.strip("\n").lower() for mot in open(chemin+"/Ressources/liste_pays.txt", encoding="utf8")])

424


In [ ]:
vecteur_lieu = moyenne_vecteurs(types_lieux)
vecteur_geo =  moyenne_vecteurs(geos)

16
15


In [ ]:
def cree_features_mention(df):
  data = []
  for i  in df.index:
    mention = df.loc[i,"mention"]
    
    if i%10 == 0 :
      print(i)
    mention= normalise_mention(mention)
    is_norme = detecte_norme(mention)
    is_lieu,is_geo,dist_lieu,dist_geo,dist_norme = contient_lieu_geo(mention,vecteur_lieu,vecteur_geo)
    #dist_lieu,dist_geo,dist_norme = distances_procedes(mention,vecteur_lieu,vecteur_geo)
    data.append([is_norme,contient_ordinal(mention),contient_superlatif(mention),is_lieu,dist_lieu,is_geo,dist_geo,dist_norme,contient_guillemet(mention)])

  return pd.DataFrame(data,index=df.index,columns=["is_norme","contient_ordinal","contient_superlatif","contient_lieu","distance_lieu","contient_geo","distance_geo","distance_norme","contient_guillemet"])

le « meilleur site d'Europe »
la « City » des Hauts-de-Seine
la « City » des Hauts-de-Seine
« coeur battant économique de la région »


In [ ]:
df_features = cree_features_mention(df)

In [ ]:
df_features.to_csv(chemin+"/Corpus/features_mentions_Defense.csv",sep=",")

In [ ]:
df_features

,is_norme,contient_ordinal,contient_superlatif,contient_lieu,distance_lieu,contient_geo,distance_geo,distance_norme,contient_guillemet
1,0,1,0,1,0.760152,1,0.272843,0.057394,0
2,1,0,0,0,0.046908,0,0.336291,1.000000,0
3,1,0,0,0,0.046908,0,0.336291,1.000000,0
4,0,0,0,1,-0.072646,1,-0.051843,-0.157264,0
5,1,0,0,0,0.046908,0,0.336291,1.000000,0
...,...,...,...,...,...,...,...,...,...
1458,1,0,0,0,0.046908,0,0.336291,1.000000,0
1459,1,0,0,0,0.046908,0,0.336291,1.000000,0
1460,0,0,0,1,0.760152,0,0.272843,0.057394,0
1461,1,0,0,0,0.046908,0,0.336291,1.000000,0
